# Scene

As all of us now, in many sitcoms and Seinfeld specifically the show is a series of scene.  
Many times, two separate scenes are completely different (for example George and Jerry are in the street, next scene they sit together in a restaurant).  
Since we except following sentences will help us better predict, we shouldn't consider different scene as following sentences.  
Moreover, we might think that the sentence timing within the scene might help the prediction, for example we remember Kramer break in to the room and the audience laughing.

After watching enough Seinfeld episodes we now that sometimes we have this _[translation music](https://www.youtube.com/watch?v=EP2GMyQdw78)_ it's not always the case, and we did see different timings of this interlude.

In [1]:
from utils import load_corpus, split_train_test
from compare_models import plot_confusion_matrix, compare_models_roc_curve
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
%load_ext autoreload
%autoreload 2

df = load_corpus()

Loading BokehJS ...

Let's start with finding the time between following sentences, we shall remember that new episodes should have time zero

In [2]:
df.loc[:, 'time_from_prev'] = np.array([0] + [df.start[i] - df.end[i - 1]
                                       if df.episode_num[i] == df.episode_num[i - 1] else 0
                                       for i in range(1, len(df.start))])


In [3]:
p = figure(plot_height=300, tools="pan,box_zoom,reset, save")

hist, edges = np.histogram(df.time_from_prev, bins=np.arange(-0.1,50,0.2))
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4)

show(p)

What do we see?  
As expected, most of the following sentences occur after < 1 sec.  
But we are interested in the long tail, it's very weird to see distance of > 40 sec, let's look at them

In [4]:
df.iloc[[j for i in df[(df.time_from_prev > 40)].index for j in [i-1, i]]].head(6)

,avg_word_length,character,end,episode_name,episode_num,is_funny,laugh_time,length,line_num,num_words,season,start,total_lines,txt,global_episode_num,time_from_prev
7147,4.000000,ELAINE,1255.228,The Smelly Car,21,True,1255.3,1.517,461,2.0,4,1253.711,475,Sauce me.,15,0.000
7148,2.800000,ELAINE,1302.299,The Smelly Car,21,False,NaN,2.125,462,5.0,4,1300.174,475,Why do we need BO?,15,44.946
11216,5.400000,GEORGE,426.385,The Barber,8,True,451.5,2.694,176,5.0,5,423.691,445,"Well, we'll straighten him out.",24,0.074
11217,3.285714,GEORGE,472.437,The Barber,8,False,NaN,1.533,177,7.0,5,470.904,445,What do you do there all day?,24,44.519
21258,4.000000,STEIN,1283.827,The Jimmy,19,True,1284.7,5.299,441,11.0,6,1278.528,454,I wanna dedicate this song to a very courageo...,44,0.741
21259,4.600000,KRAMER,1390.368,The Jimmy,19,False,NaN,2.398,442,5.0,6,1387.970,454,"Hey, got the new Penthouse.",44,104.143


After watching some episodes that's look all right.  
It's really a new scene, and it there is that match seconds without speaking.  
  
  
We prefer to have false negative than false positive --> we prefer to have two scenes together than to break in a middle of a scene, so we will pick the threshold a bit high, let's say 5 seconds

In [5]:
df.episode_num.sample(1).values[0]

8

In [6]:
# df.loc[:, ['new_scene']] = df['time_from_prev'] > 5
# # Other than that, if it's a new episode we want new scene, used heuristic for that
# df.loc[(df.time_from_prev == 0) & (df.start <= 2), ['new_scene']] = True

df[(df.episode_num == df.episode_num.sample(1).values[0]) & (df.season == df.season.sample(1).values[0])]

,avg_word_length,character,end,episode_name,episode_num,is_funny,laugh_time,length,line_num,num_words,season,start,total_lines,txt,global_episode_num,time_from_prev
9091,6.000000,JERRY,1.996,The Sniffing Accountant,4,False,NaN,1.328,0,3.0,5,0.668,514,"To me, government...",20,0.000
9092,5.600000,JERRY,5.266,The Sniffing Accountant,4,True,2.7,3.251,1,5.0,5,2.015,514,is basically parents for adults.,20,0.019
9093,4.166667,JERRY,9.104,The Sniffing Accountant,4,False,NaN,3.297,2,11.0,5,5.807,514,Especially the IRS. The IRS is like Ward and ...,20,0.541
9094,3.625000,JERRY,12.305,The Sniffing Accountant,4,True,12.4,2.694,3,7.0,5,9.611,514,and we're all Wally and the Beaver.,20,0.507
9095,4.857143,JERRY,14.345,The Sniffing Accountant,4,False,NaN,1.965,4,6.0,5,12.380,514,And your accountant is Eddie Haskell...,20,0.075
9096,3.916667,JERRY,17.211,The Sniffing Accountant,4,True,17.0,2.796,5,11.0,5,14.415,514,showing you all these neat tricks to get away...,20,0.070
9097,5.166667,JERRY,19.544,The Sniffing Accountant,4,False,NaN,1.692,6,6.0,5,17.852,514,"That's fine, unless you get audited.",20,0.641
9098,3.333333,JERRY,22.246,The Sniffing Accountant,4,False,NaN,2.625,7,11.0,5,19.621,514,Then you don't want some wise guy in a suit g...,20,0.077
9099,3.888889,JERRY,25.722,The Sniffing Accountant,4,True,25.3,3.399,8,8.0,5,22.323,514,"""You have a very lovely office here, sir.""",20,0.077
9100,4.153846,JERRY,29.225,The Sniffing Accountant,4,True,29.5,3.365,9,12.0,5,25.860,514,Because jail is the government's way of sendi...,20,0.138


Not fantastic, we still see FP